## categories列の特徴量を作成する

In [1]:
import pandas as pd
import scipy as sc
import numpy as np
import sklearn
import pickle
import pathlib as Path
import matplotlib.pyplot as plt
import japanize_matplotlib
import seaborn as sb
sb.set(font='IPAexGothic')

import multiprocessing
import itertools
import collections
import datetime
import gc

from tqdm.notebook import tqdm

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

data_path = Path.Path("../data")
img_path = Path.Path("../img")
result_path = Path.Path("../result")

prefix = 'ana006'

## データ読み込み

In [2]:
train_df = pd.read_pickle(data_path / 'train_df.pkl')

train_df

,id,submitter,authors,title,comments,journal-ref,doi,report-no,categories,license,abstract,versions,update_date,authors_parsed,doi_cites,cites
0,hep-ph/9902295,Michael Kraemer,Mark E. Hayes (University College London) and ...,Heavy-Flavour Production at HERA,"LaTeX, 21 pages, 13 Postscript figures. Summar...","J.Phys.G25:1477-1493,1999",10.1088/0954-3899/25/7/332,"CERN-TH/99-30, UCL/HEP 99-03",hep-ph hep-ex,None,We review the theoretical and experimental s...,"[[{'version': 'v1', 'created': 'Wed, 10 Feb 19...",2008-11-26,"[[Hayes, Mark E., , University College London]...",1,NaN
1,1403.7138,Aigen Li,"Qi Li, S.L. Liang, Aigen Li (University of Mis...",Spectropolarimetric Constraints on the Nature ...,"5 pages, 2 figures; accepted for publication i...",None,10.1093/mnrasl/slu021,None,astro-ph.GA,http://arxiv.org/licenses/nonexclusive-distrib...,While it is well recognized that interstella...,"[[{'version': 'v1', 'created': 'Thu, 27 Mar 20...",2015-06-19,"[[Li, Qi, , University of Missouri], [Liang, S...",8,7.0
2,1405.5857,Michael Mortonson,"Michael J. Mortonson, Uro\v{s} Seljak",A joint analysis of Planck and BICEP2 B modes ...,"13 pages, 4 figures; submitted to JCAP; refere...",JCAP10(2014)035,10.1088/1475-7516/2014/10/035,None,astro-ph.CO gr-qc hep-ph hep-th,http://arxiv.org/licenses/nonexclusive-distrib...,We analyze BICEP2 and Planck data using a mo...,"[[{'version': 'v1', 'created': 'Thu, 22 May 20...",2014-10-17,"[[Mortonson, Michael J., ], [Seljak, Uroš, ]]",122,188.0
3,1807.01034,Evangelos Thomas Karamatskos,"Evangelos T. Karamatskos, Sebastian Raabe, Ter...",Molecular movie of ultrafast coherent rotation...,9 Figures,"Nat Commun 10, 3364 (2019)",10.1038/s41467-019-11122-y,None,physics.chem-ph physics.atom-ph quant-ph,http://arxiv.org/licenses/nonexclusive-distrib...,Recording molecular movies on ultrafast time...,"[[{'version': 'v1', 'created': 'Tue, 3 Jul 201...",2020-05-19,"[[Karamatskos, Evangelos T., ], [Raabe, Sebast...",6,8.0
4,1905.05921,Juanjuan Gu,Juanjuan Gu and Yun Jing,A Modified Mixed Domain Method for Modeling Ac...,None,None,10.1121/10.0001454,None,physics.med-ph physics.comp-ph,http://arxiv.org/licenses/nonexclusive-distrib...,"In this paper, phase correction and amplitud...","[[{'version': 'v1', 'created': 'Wed, 15 May 20...",2020-07-15,"[[Gu, Juanjuan, ], [Jing, Yun, ]]",0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
851519,1708.06097,Haim Diamant,Rony Granek and Haim Diamant,Membrane undulations in a structured fluid: Un...,10 pages,"Eur. Phys. J. E 41, 1 (2018)",10.1140/epje/i2018-11607-x,None,cond-mat.soft,http://arxiv.org/licenses/nonexclusive-distrib...,The dynamics of membrane undulations inside ...,"[[{'version': 'v1', 'created': 'Mon, 21 Aug 20...",2018-02-20,"[[Granek, Rony, ], [Diamant, Haim, ]]",0,NaN
851520,1904.07627,Chonglong Liu,"C. L. Liu, Xiao-Dong Yu, D. M. Tong",Flag Additivity in Quantum Resource Theories,6 pages,"Phys. Rev. A 99, 042322(2019)",10.1103/PhysRevA.99.042322,None,quant-ph,http://arxiv.org/licenses/nonexclusive-distrib...,Quantum resource theories offer a powerful f...,"[[{'version': 'v1', 'created': 'Tue, 16 Apr 20...",2019-04-17,"[[Liu, C. L., ], [Yu, Xiao-Dong, ], [Tong, D. ...",2,NaN
851521,1507.01140,Othman Benomar,"O. Benomar, M. Takata, H. Shibahashi, T. Ceill...",Nearly-uniform internal rotation of solar-like...,Accepted to MNRAS,None,10.1093/mnras/stv1493,None,astro-ph.SR,http://arxiv.org/licenses/nonexclusive-distrib...,The rotation rates in the deep interior and ...,"[[{'version': 'v1', 'created': 'Sat, 4 Jul 201...",2015-08-06,"[[Benomar, O., ], [Takata, M., ], [Shibahashi,...",42,NaN
851522,2006.03674,Sukriti Manna,"Troy D Loeffler, Sukriti Manna, Tarak K Patra,...",Active Learning A Neural Network Model For Gol...,None,None,10.1002/cctc.202000774,None,physics.comp-ph cond-mat.mtrl-sci,http://arxiv.org/licenses/nonexclusive-distrib...,Small metal clusters are of fundamental scie...,"[[{'version': 'v1', 'created': 'Fri, 5 Jun 202..."

## 特徴量作成
> categories列をエンコードできるかどうか考える

In [4]:
train_df['categories'].value_counts(dropna=False)

astro-ph                                       61053
hep-ph                                         48319
hep-th                                         38078
quant-ph                                       32610
cond-mat.mes-hall                              22386
                                               ...  
cs.NA cs.DS math.NA stat.CO stat.ML                1
hep-th cond-mat.dis-nn nlin.CD quant-ph            1
hep-th cs.SC hep-ph math-ph math.CA math.MP        1
nlin.CG cond-mat.stat-mech cs.CC nlin.AO           1
q-bio.QM stat.AP stat.ML                           1
Name: categories, Length: 32734, dtype: int64

> 欠損はなさそう、一部半角スペースで複数に区切られている

In [11]:
# スペース区切りでlistにする
categories_list = train_df['categories'].apply(lambda x: x.split(' '))

# 種類数を見てみる (one hot encodingしたときに、許容できる次元数となるか？)
unique_categories = set(itertools.chain.from_iterable(categories_list))

len(unique_categories)
unique_categories

176

{'acc-phys',
 'adap-org',
 'alg-geom',
 'ao-sci',
 'astro-ph',
 'astro-ph.CO',
 'astro-ph.EP',
 'astro-ph.GA',
 'astro-ph.HE',
 'astro-ph.IM',
 'astro-ph.SR',
 'atom-ph',
 'bayes-an',
 'chao-dyn',
 'chem-ph',
 'cmp-lg',
 'comp-gas',
 'cond-mat',
 'cond-mat.dis-nn',
 'cond-mat.mes-hall',
 'cond-mat.mtrl-sci',
 'cond-mat.other',
 'cond-mat.quant-gas',
 'cond-mat.soft',
 'cond-mat.stat-mech',
 'cond-mat.str-el',
 'cond-mat.supr-con',
 'cs.AI',
 'cs.AR',
 'cs.CC',
 'cs.CE',
 'cs.CG',
 'cs.CL',
 'cs.CR',
 'cs.CV',
 'cs.CY',
 'cs.DB',
 'cs.DC',
 'cs.DL',
 'cs.DM',
 'cs.DS',
 'cs.ET',
 'cs.FL',
 'cs.GL',
 'cs.GR',
 'cs.GT',
 'cs.HC',
 'cs.IR',
 'cs.IT',
 'cs.LG',
 'cs.LO',
 'cs.MA',
 'cs.MM',
 'cs.MS',
 'cs.NA',
 'cs.NE',
 'cs.NI',
 'cs.OH',
 'cs.OS',
 'cs.PF',
 'cs.PL',
 'cs.RO',
 'cs.SC',
 'cs.SD',
 'cs.SE',
 'cs.SI',
 'cs.SY',
 'dg-ga',
 'econ.EM',
 'econ.GN',
 'econ.TH',
 'eess.AS',
 'eess.IV',
 'eess.SP',
 'eess.SY',
 'funct-an',
 'gr-qc',
 'hep-ex',
 'hep-lat',
 'hep-ph',
 'hep-th',
 'm

> 176ね～

In [16]:
%%time

# get_dummiesを使ってcategories列をonehot encoding ※それなりにメモリを食うかも、スパース化を検討してもよい
categories_x_df = pd.get_dummies(train_df['categories'].apply(lambda x: pd.Series(x.split(' '))).stack()).sum(level=0)

categories_x_df

Wall time: 3min 13s


,acc-phys,adap-org,alg-geom,ao-sci,astro-ph,astro-ph.CO,astro-ph.EP,astro-ph.GA,astro-ph.HE,astro-ph.IM,...,q-fin.TR,quant-ph,solv-int,stat.AP,stat.CO,stat.ME,stat.ML,stat.OT,stat.TH,supr-con
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
851519,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
851520,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
851521,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
851522,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [21]:
# ちゃんと値入ってる?
categories_x_df.iloc[0]['hep-ph']
categories_x_df.iloc[0]['hep-ex']

categories_x_df.iloc[0].sum()

1

1

2

In [22]:
# 特徴量保存
categories_x_df.to_pickle(data_path / ('%s_x.pkl' % prefix))